In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import json
import csv
from pandas import DataFrame
import time
import gc

import cv2

from IPython.display import Image
from IPython.core.display import HTML

from scipy.sparse import csr_matrix

%matplotlib inline 

import os
print(os.listdir("."))

['.git', '.gitignore', '.ipynb_checkpoints', 'Classification.ipynb', 'data', 'feature_extractor.py', 'feature_extractor_linux.py', 'keras_test.py', 'LICENSE', 'out.csv', 'README.md', 'rescale.py', 'submission.csv', 'test.csv', 'test.json', 'train.json', 'Untitled.ipynb', 'validation.json']


In [5]:
import glob
import pickle

FEATURES_LOCATION = './data/features/'
F_CORE = 'cnn_features_'

def get_label_from_path(file):
    return file.split('\\')[1].split('.')[0]

def load_data(mode):
    if(mode == 'test'):
        pickle_path = F_CORE + mode
        data = pickle.load(open(FEATURES_LOCATION + pickle_path + '.pkl', 'rb'))
        to_return = {}
        for key, value in list(data.items()):
            to_return[get_label_from_path(key)] = value.reshape(1,-1)
        return to_return, None
    
    pickle_path = F_CORE + mode + '_'
    
    data = {}
    for i in range(1,129):
        data[i] = pickle.load(open(FEATURES_LOCATION + pickle_path + str(i) + '.pkl', 'rb'))
        
    X = []
    y = []
    for key, value in list(data.items()):
        the_class = key
        features = np.array(list(value.values()))
        for feature in features:
            y.append(the_class)
            X.append(feature)

    return np.array(X), np.array(y)

In [6]:
X, y = load_data('train')

In [7]:
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MultiLabelBinarizer 
# y_mlb = MultiLabelBinarizer().fit_transform(y)
# print(y)

from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold()
skf.get_n_splits(X, y)

StratifiedKFold(n_splits=3, random_state=None, shuffle=False)
# for train_index, test_index in skf.split(X, y):
#     print("TRAIN:", train_index, "TEST:", test_index)
#     X_train, X_test = X[train_index], X[test_index]
#     y_train, y_test = y[train_index], y[test_index]

mlp = MLPClassifier(hidden_layer_sizes=(200, 100), max_iter=200, alpha=1e-4,
                    solver='sgd', verbose=10, tol=1e-4, random_state=1,
                    learning_rate='adaptive', learning_rate_init=.1,
                   early_)
mlp.fit(X, y)

StratifiedKFold(n_splits=3, random_state=None, shuffle=False)
for train_index, test_index in skf.split(X, y):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    print("Training set score: %f" % mlp.score(X_train, y_train))
    print("Test set score: %f" % mlp.score(X_test, y_test))
    


Iteration 1, loss = 1.42169598
Iteration 2, loss = 1.07068397
Iteration 3, loss = 0.96465872
Iteration 4, loss = 0.89250515
Iteration 5, loss = 0.84201521
Iteration 6, loss = 0.80044844
Iteration 7, loss = 0.76176381
Iteration 8, loss = 0.72834389
Iteration 9, loss = 0.70717050
Iteration 10, loss = 0.68041461
Iteration 11, loss = 0.65688303
Iteration 12, loss = 0.64092608
Iteration 13, loss = 0.61912717
Iteration 14, loss = 0.61550232
Iteration 15, loss = 0.59147971
Iteration 16, loss = 0.58939011
Iteration 17, loss = 0.56983562
Iteration 18, loss = 0.56280231
Iteration 19, loss = 0.55121680
Iteration 20, loss = 0.54297954
Iteration 21, loss = 0.53507924
Iteration 22, loss = 0.53226530
Iteration 23, loss = 0.51224345
Iteration 24, loss = 0.51605744
Iteration 25, loss = 0.50099846
Iteration 26, loss = 0.49769710
Iteration 27, loss = 0.48720257
Iteration 28, loss = 0.49002446
Iteration 29, loss = 0.48366202
Iteration 30, loss = 0.47170707
Iteration 31, loss = 0.46724566
Iteration 32, los

NameError: name 'clf' is not defined

In [8]:
from sklearn.externals import joblib
joblib.dump(mlp, 'mlp_200_100.pkl') 

['mlp_200_100.pkl']

In [ ]:
mlp2 = MLPClassifier(hidden_layer_sizes=(400, 200), max_iter=1000, alpha=1e-4,
                    solver='sgd', verbose=10, tol=1e-4, random_state=1,
                    learning_rate='adaptive', learning_rate_init=.1)
mlp2.fit(X, y)

In [10]:
X_val, y_val = load_data('valid')

In [11]:
mlp.score(X_val, y_val)

0.7262640672055793

In [151]:
X_test, _ = load_data('test')

In [152]:
predictions = {}
for index, feature_vector in X_test.items():
    predictions[int(index)] = mlp.predict(feature_vector)[0]

In [153]:
from collections import Counter
counted = Counter(predictions.values())
most_common_class = counted.most_common()[0][0]

for index in range(1, 12801):
    if(index not in predictions.keys()):
        predictions[index] = most_common_class

In [154]:
ids = []
values = []
for key, value in predictions.items():
    ids.append(key)
    values.append(value)
    
out_dict = {}
out_dict['id'] = ids
out_dict['predicted'] = values

In [155]:
keys = sorted(out_dict.keys())

with open('out.csv', 'w') as csv:
    # Write keys    
    csv.write(','.join([k for k in keys]) + '\n')

    # Assume all values of dict are equal
    for i in range(len(out_dict[keys[0]])):
        csv.write(','.join([FMT % out_dict[k][i] for k in keys]) + '\n')